### Import packages and libraries

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img ,img_to_array
from sklearn.metrics import classification_report, confusion_matrix

### Data loading and augmentation

In [2]:
# Define path
train_dir="new_train"
test_dir="new_test"

batch_size=32
image_size=(28,28)
# Use ImageDataGenerator for data augmentation and preprocessing
datagen =ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)
# Use flow_from_directory to load images from directory
train_generator=datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator=datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2280 images belonging to 57 classes.
Found 855 images belonging to 57 classes.


### Model Architecture

In [3]:
#  Built CNN model using keras
model = Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(28,28,3)),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(57,activation='softmax')
])

### Model compiling

In [4]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Model training

In [5]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50)

Epoch 1/50


71/71 [==============================] - 33s 455ms/step - loss: 3.9210 - accuracy: 0.0565 - val_loss: 3.2613 - val_accuracy: 0.1743
Epoch 2/50
71/71 [==============================] - 6s 80ms/step - loss: 2.6467 - accuracy: 0.2585 - val_loss: 2.0149 - val_accuracy: 0.4023
Epoch 3/50
71/71 [==============================] - 5s 70ms/step - loss: 1.7481 - accuracy: 0.4835 - val_loss: 1.2658 - val_accuracy: 0.6094
Epoch 4/50
71/71 [==============================] - 5s 67ms/step - loss: 1.2110 - accuracy: 0.6286 - val_loss: 0.9054 - val_accuracy: 0.7216
Epoch 5/50
71/71 [==============================] - 5s 67ms/step - loss: 0.9582 - accuracy: 0.6962 - val_loss: 0.7601 - val_accuracy: 0.7520
Epoch 6/50
71/71 [==============================] - 5s 70ms/step - loss: 0.7683 - accuracy: 0.7562 - val_loss: 0.7169 - val_accuracy: 0.7930
Epoch 7/50
71/71 [==============================] - 5s 69ms/step - loss: 0.6672 - accuracy: 0.7829 - val_loss: 0.5102 - val_accuracy: 0.8269
Epoch 8/5

### Model evaluation

In [6]:
test_loss,test_acc=model.evaluate(test_generator,verbose=2)
print('Test accuracy:',test_acc)

27/27 - 1s - loss: 0.1369 - accuracy: 0.9532 - 1s/epoch - 46ms/step
Test accuracy: 0.9532163739204407


### Model summery

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 1, 1, 128)         0

### Line plot Training val_accuracy vs accuracy 

In [ ]:
epochs =[i for i in range(0,50)]
plt.plot(epochs, history.history["accuracy"], color="red", label="Training Accuracy")
plt.plot(epochs, history.history["val_accuracy"],color="blue",label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Visualisation of Accuracy Result")
plt.legend()
plt.show()

### Line plot Training val_loss vs loss 

In [ ]:
epochs =[i for i in range(0,50)]
plt.plot(epochs, history.history["loss"], color="red", label="Training Accuracy")
plt.plot(epochs, history.history["val_loss"],color="blue",label="Validation Accuracy")
plt.xlabel("loss")
plt.ylabel("val_loss")
plt.title("Visualisation of loss Result")
plt.legend()
plt.show()

### Predict, confusion matrix and Classification report 

In [9]:
# Generate predictions
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

# Confusion matrix and classification report
print('Confusion Matrix')
cm=confusion_matrix(test_generator.classes, y_pred) 
print(cm)

print('Classification Report')
target_names = list(test_generator.class_indices.keys())
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

27/27 [==============================] - 1s 44ms/step
Confusion Matrix
[[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 1 1]
 [0 0 1 ... 0 1 0]
 ...
 [1 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 1]]
Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        15
           1       0.00      0.00      0.00        15
          10       0.07      0.07      0.07        15
          11       0.00      0.00      0.00        15
          12       0.00      0.00      0.00        15
          13       0.12      0.13      0.12        15
          14       0.00      0.00      0.00        15
          15       0.00      0.00      0.00        15
          16       0.00      0.00      0.00        15
          17       0.00      0.00      0.00        15
          18       0.00      0.00      0.00        15
          19       0.00      0.00      0.00        15
           2       0.00      0.00      0.00        15
          20       0.08      0

### Heatmap

In [4]:
# ### Confusion Matrix Visualisation 
# plt.figure(figsize=(40,40))
# sns.heatmap(cm,annot=True)
# plt.xlabel("Predicted Class" , fontsize=20)
# plt.ylabel("Actual Class",fontsize=20)
# plt.title("Prediction Confusion Matrix" , fontsize=30)
# plt.show()

### Save training History

In [11]:
import json
with open("history.json", "w") as f:
    json.dump(history.history,f)

### Save model

In [21]:
model.save("model.keras")

### Load model

In [23]:
model = load_model('model.keras')

In [24]:
image_path = '\\Python\\Odia_project\\new_data\\0\\0.jpg'

### Output prediction

In [25]:
image =load_img(image_path,target_size=(50,50))
input_arr=img_to_array(image)
input_arr=np.array([input_arr])  ## Converting single image to  batch
print(input_arr.shape)
predictions = model.predict(input_arr)
print(predictions,predictions.shape)

(1, 50, 50, 3)
1/1 [==============================] - 0s 83ms/step
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]] (1, 57)


### ALternative way to load and predict image

In [ ]:
# image_size=(50,50)
# image = cv2.imread(image_path)
# image = cv2.resize(image, image_size)
# print(image.shape)
# predictions = model.predict(image.reshape(1,50,50,3))
# print(predictions,predictions.shape)

### Output Class

In [26]:
result_index = np.argmax(predictions)
result_index

0

In [27]:
df=pd.read_csv('odia_label.csv')
df

,class,label,class.1,label.1,class.2,label.2,class.3,label.3,class.4,label.4,class.5,label.5,class.6,label.6,class.7,label.7,class.8,label.8,class.9,label.9
0,0,ଅ,6,ଋ,12,କ,18,ଛ,24,ଡ,30,ଧ,36,ମ,42,ସ,48,୧,54.0,୭
1,1,ଆ,7,ୠ,13,ଖ,19,ଜ,25,ଢ,31,ନ,37,ଯ,43,ହ,49,୨,55.0,୮
2,2,ଇ,8,ଏ,14,ଗ,20,ଝ,26,ଣ,32,ପ,38,ର,44,କ୍ଷ,50,୩,56.0,୯
3,3,ଈ,9,ଐ,15,ଘ,21,ଞ,27,ତ,33,ଫ,39,ଳ,45,ୟ,51,୪,NaN,NaN
4,4,ଉ,10,ଓ,16,ଙ,22,ଟ,28,ଥ,34,ବ,40,ଶ,46,ଲ,52,୫,NaN,NaN
5,5,ଊ,11,ଔ,17,ଚ,23,ଠ,29,ଦ,35,ଭ,41,ଷ,47,୦,53,୬,NaN,NaN


In [28]:
df.iloc[result_index]['label']

'ଅ'